In [1]:
import azureml.core
from azureml.core import Experiment, Workspace

# Check core SDK version number
print("This notebook was created using version 1.0.2 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")
print("")


ws = Workspace.from_config()
# print('Workspace name: ' + ws.name, 
#       'Azure region: ' + ws.location, 
#       'Subscription id: ' + ws.subscription_id, 
#       'Resource group: ' + ws.resource_group, sep='\n')

This notebook was created using version 1.0.2 of the Azure ML SDK
You are currently using version 1.1.5 of the Azure ML SDK



In [2]:
#!pip install nltk
#!pip install xgboost
#!pip install textblob 
import numpy as np
import pandas as pd
import os
import nltk
#from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn import linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

from sklearn.metrics import accuracy_score
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from numpy import array
#from numpy import asarray

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from azureml.core import Workspace, Dataset

#subscription_id,resource_group, workspace_name are removed for security purpose

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='mtsamples')
data = dataset.to_pandas_dataframe()

In [4]:
data = data[['Group','transcription']]

In [5]:
data.head()

,Group,transcription
0,Allergy an immunology,"SUBJECTIVE:, This 23-year-old white female pr..."
1,NA,"PAST MEDICAL HISTORY:, He has difficulty climb..."
2,NA,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ..."
3,Cardiology,"2-D M-MODE: , ,1. Left atrial enlargement wit..."
4,Cardiology,1. The left ventricular cavity size and wall ...


In [6]:
df = data

In [7]:
df['transcription']=df['transcription'].fillna(df['transcription'].mode()[0])

In [8]:
df.isnull().sum()

Group            29
transcription     0
dtype: int64

In [9]:
df.dropna(inplace=True)

In [10]:
df.describe()

,Group,transcription
count,4999,4999
unique,33,2363
top,Surgery,
freq,1093,37


In [11]:
df['Group'] = df[df['Group']!='NA'] 

In [12]:
df = df.dropna().reset_index(drop=True)

In [ ]:
#preprocessing
df['transcription']=df['transcription'].apply(lambda x:" ".join(x.lower() for x in x.split()))
df['transcription']=df['transcription'].str.replace('[^\w\s]','')
stop=stopwords.words('english')
df['transcription']=df['transcription'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))
df['transcription'].head()

In [15]:
df['transcription']=df['transcription'].apply(lambda x:" ".join(x for x in x.split(' ')[:500]))

In [16]:
speciality=['Anesthesiology','Psychiatry','Dermatology','Others','Physical medicine / rehabilitation','Plastic surgery','Other',
            'Endocrinology, diabetes and metabolism','Pathology','Allergy an immunology']

In [17]:
remove_speciality=['Otolaryngology','Neurological surgery','Oncology (cancer)','Ophthalmology','Pediatrics']

In [18]:
df['Group']= df['Group'].str.strip()

In [ ]:
df['Group'].value_counts()

In [20]:
df=df[~df['Group'].isin(remove_speciality)]

In [21]:
df['Group']=df['Group'].replace(to_replace=speciality,value="Others")

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
df['totalwords'] = [len(x.split()) for x in df['transcription'].tolist()]

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
df=df.loc[df['totalwords']>20]

In [24]:
df['totalwords'].min()

21

In [25]:
df = df[df['Group']!='']

In [26]:
df = df.reset_index(drop=True)

In [ ]:
df['Group'].value_counts()

In [28]:
data_x=df['transcription']
data_y=df['Group']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.30, random_state = 42)

from sklearn import model_selection, preprocessing
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(y_train)
#encoder_name_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
#print(encoder_name_mapping)
valid_y = encoder.fit_transform(y_test)

In [ ]:
encoder_name_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(encoder_name_mapping)

In [30]:
import joblib

In [31]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                             max_features=5000, stop_words='english')
tfidf_vect=tfidf_vect.fit(data_x)
model_name = 'tfid_vectors.pkl'

joblib.dump(value = tfidf_vect,filename = model_name)

X_train_tfidf =  tfidf_vect.fit_transform(X_train)
X_test_tfidf =  tfidf_vect.transform(X_test)

In [ ]:
# Get an experiment object from Azure Machine Learning
experiment = Experiment(workspace=ws, name="special_")
# Create a run object in the experiment
run =  experiment.start_logging()

In [33]:
#naive bayes

NB_model=naive_bayes.MultinomialNB()
NB_model.fit(X_train_tfidf, train_y)
preds = NB_model.predict(X_test_tfidf)

print('accuracy of the model is ',accuracy_score(valid_y,preds))
run.log('accuracy',accuracy_score(valid_y,preds))

# Save the model to the outputs directory for capture
model_file_name = 'outputs/spe_model.pkl'

joblib.dump(value = NB_model, filename = model_file_name)

# upload the model file explicitly into artifacts 
run.upload_file(name = model_file_name, path_or_stream = model_file_name)

# Complete the run
run.complete()
print("the run has been completed")

accuracy of the model is  0.4872549019607843
the run has been completed


In [ ]:
#here the two saved models are in outputs and in model_path, the folder name is provided.

In [34]:
model = run.register_model(model_name='speciality_classification', model_path='outputs/')

In [35]:
from azureml.core.model import Model
models = Model.list(ws, name='speciality_classification')
for m in models:
    print(m.name, m.version)

speciality_classification 3


In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.image import ContainerImage
from azure.core import Environment

# Create an empty conda environment and add the scikit-learn package
env = CondaDependencies()
env.add_conda_package("scikit-learn")

# Display the environment
print(env.serialize_to_string())

# Write the environment to disk
with open("myenv.yml","w") as f:
    f.write(env.serialize_to_string())

# Create a configuration object indicating how our deployment container needs to be created
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                    runtime="python", 
                                    conda_file="myenv.yml")

In [49]:
image_config

In [42]:
#target compute
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={'speciality': 'oncology'}, 
                                               description='treats cancer')

In [43]:
#deploy webservice

from azureml.core.webservice import Webservice

# Create the webservice using all of the precreated configurations and our best model
service = Webservice.deploy_from_model(name='svc-specialty',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config,
                                       workspace=ws)

# Wait for the service deployment to complete while displaying log output
service.wait_for_deployment(show_output=True)


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: deploy_from_model has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  # Remove the CWD from sys.path while we load stuff.
ERROR - Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 7a5cfe29-0f3f-4de1-8b1f-911b2c90b6a2
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice."
}

ERROR - Ser

Creating image
Running..................................................................
Succeeded
Image creation operation finished for image svc-specialty:1, operation "Succeeded"
Running...............................................................................................................................................................................................................................
TimedOut


WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 7a5cfe29-0f3f-4de1-8b1f-911b2c90b6a2
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: 7a5cfe29-0f3f-4de1-8b1f-911b2c90b6a2\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"DeploymentTimedOut\

In [53]:
service.state

'Unhealthy'

In [52]:
service.get_logs()

ERROR - Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Thu, 16 Apr 2020 11:55:56 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-client-request-id': '134c49b273c5417682442b6cd4bcce03', 'x-ms-client-session-id': '6202233d-7103-45fc-acc9-451b393d53aa', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found","details":[{"code":"NoSuchService","message":"There is no service in Subscription: 8ef809cc-7c53-43fc-abe3-e3af5840fafd, ResourceGroup: omnipresence-poc, Workspace: omnipresence-aa-poc with name: svc-specialty"}]}'



WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Thu, 16 Apr 2020 11:55:56 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-client-request-id': '134c49b273c5417682442b6cd4bcce03', 'x-ms-client-session-id': '6202233d-7103-45fc-acc9-451b393d53aa', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found","details":[{"code":"NoSuchService","message":"There is no service in Subscription: 8ef809cc-7c53-43fc-abe3-e3af5840fafd, ResourceGroup: omnipresence-poc, Workspace: omnipresence-aa-poc with name: svc-specialty"}]}'
	InnerException None


In [48]:
from azureml.core.model import Model
import logging
logging.basicConfig(level=logging.DEBUG)
print(Model.get_model_path(model_name='outputs/tfid_vectors.pkl'))

outputs/tfid_vectors.pkl


In [44]:
a = service.get_logs()